In [1]:
import findspark
findspark.init('/home/hadoop/spark-2.2.2-bin-hadoop2.7')
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
conf = SparkConf().setMaster("spark://master:7077")
sc = SparkContext.getOrCreate()
sqlcontext = SQLContext(sc)

In [100]:
dist_df=sqlcontext.read.option("header","true").csv('hdfs://master:9000/user/hadoop/dist.csv')
dist_df.show()

+---+------+------+------------+
|_c0|     0|     1|           2|
+---+------+------+------------+
|  0|186814|742349| 0.182584116|
|  1|563786|742349| 0.197973076|
|  2|680193|742349|  0.25157314|
|  3|685989|742349| 0.267482115|
|  4|703852|742349| 0.104270999|
|  5|703856|742349| 0.187936461|
|  6|720190|742349| 0.215755544|
|  7|728751|742349|0.0790675577|
|  8|260510|742349| 0.213686329|
|  9|312979|742349| 0.411003595|
| 10|325631|742349| 0.281914946|
| 11|644182|742349|0.0844898709|
| 12|654138|742349| 0.124556778|
| 13|729047|742349| 0.201668716|
| 14|731939|742349|0.0713034077|
| 15|737378|742349| 0.260800984|
| 16| 22896|742349| 0.141093713|
| 17|641253|742349|   0.1311179|
| 18|642598|742349| 0.266048767|
| 19|713975|742349| 0.176263051|
+---+------+------+------------+
only showing top 20 rows



In [101]:
import pyspark.sql.functions as F

dist_df = dist_df.select(F.col("_c0").alias("index"), F.col("0").alias("comparison"), F.col("1").alias("webtoon"), F.col("2").alias("distance"))
dist_df.orderBy("distance", ascending=True).show(5)

+-----+----------+-------+------------+
|index|comparison|webtoon|    distance|
+-----+----------+-------+------------+
|  335|    708453| 742349|0.0460525463|
|  305|    651673| 742349|0.0486622192|
|  239|    732256| 742349|0.0492225898|
|  449|    740034| 742349|0.0518217632|
|  302|     58293| 742349|0.0518996559|
+-----+----------+-------+------------+
only showing top 5 rows



In [102]:
dict_df=sqlcontext.read.json('hdfs://master:9000/user/hadoop/dict.json')

In [105]:
df_dict = dict_df.take(1)[0].asDict()

def add_descriptions(in_dict):
    def f(x):
        return in_dict.get(x)
    return F.udf(f)

matched_df = dist_df.withColumn(
    "comparison",
    add_descriptions(df_dict)(dist_df.comparison)
)

matched_df = matched_df.withColumn(
    "webtoon",
    add_descriptions(df_dict)(matched_df.webtoon)
)

matched_df.show()

matched_df.orderBy("distance", ascending=True).show(5)

+-----+----------+---------+------------+
|index|comparison|  webtoon|    distance|
+-----+----------+---------+------------+
|    0|   심부름센터 K|나의 첫번째 새벽| 0.182584116|
|    1|OH, MY ...|나의 첫번째 새벽| 0.197973076|
|    2|한국만화거장전...|나의 첫번째 새벽|  0.25157314|
|    3|        폰령|나의 첫번째 새벽| 0.267482115|
|    4|  바른연애 길잡이|나의 첫번째 새벽| 0.104270999|
|    5|   8월의 눈보라|나의 첫번째 새벽| 0.187936461|
|    6|      머니게임|나의 첫번째 새벽| 0.215755544|
|    7|      괴물신부|나의 첫번째 새벽|0.0790675577|
|    8|MODERN ...|나의 첫번째 새벽| 0.213686329|
|    9|힘내요 일본!...|나의 첫번째 새벽| 0.411003595|
|   10|     시간의 섬|나의 첫번째 새벽| 0.281914946|
|   11|     지새는 달|나의 첫번째 새벽|0.0844898709|
|   12|은주의 방 2~3부|나의 첫번째 새벽| 0.124556778|
|   13|       공유몽|나의 첫번째 새벽| 0.201668716|
|   14|     나의 우주|나의 첫번째 새벽|0.0713034077|
|   15|      블루투스|나의 첫번째 새벽| 0.260800984|
|   16|     핑크레이디|나의 첫번째 새벽| 0.141093713|
|   17|    외모지상주의|나의 첫번째 새벽|   0.1311179|
|   18|    조선왕조실톡|나의 첫번째 새벽| 0.266048767|
|   19|     데드라이프|나의 첫번째 새벽| 0.176263051|
+-----+----------+---------+------